## Alpha Zero for Gomoku

In this seminar we look at an implementation of [Alpha Zero](https://kstatic.googleusercontent.com/files/2f51b2a749a284c2e2dfa13911da965f4855092a179469aedd15fbe4efe8f8cbf9c515ef83ac03a6515fa990e6f85fd827dcd477845e806f23a17845072dc7bd). Technically, we look at its predecessor, a version specialized to Go called [AlphaGo Zero](https://www.nature.com/articles/nature24270).

In the lecture we spoke about AlphaGo. A review of AlphaGo pipeline: 

## AlphaGo Pipeline

<center>
  <figure>
    <img src="images/pipeline1.png" width=800>
    <figcaption>
      Policy and value networks.
    </figcaption>
  </figure>
</center>

<center>
  <figure>
    <img src="images/pipeline2.png" width=800>
    <figcaption>
      An Illustration of MCTS.
    </figcaption>
  </figure>
</center>

## AlphaGo Zero

### Differences between AlphaGo and AlphaGo Zero

* AlphaGo uses supervised learning from human expert moves whereas AlphaGo Zero is solely based on reinforcement learning, without human data, guidance or domain knowledge beyond game rules.
* AlphaGo trains the policy and value networks separately whereas AlphaGo Zero uses a single neural network.
* AlphaGo Zero uses Monte-Carlo tree search as a policy improvement step to simulate data trajectories (according to UCB) and Monte Carlo evaluation as a policy evaluation step to update the policy and value network. It repeats this procedure to improve the policy and value network (similar to policy iteration). 

### Monte-Carlo-Tree-Search

<center>
  <figure>
    <img src="images/MCTS.png" width=800>
    <figcaption>
      Taken from DeepMind. Great depiction of MCTS.
    </figcaption>
  </figure>
</center>

Once the Tree Search is finished we collect tree search values $z$ and tree search probabilities $\pi$ for all the nodes in the search tree.

### Self-Play

<center>
  <figure>
    <img src="images/self_play.png" width=800>
    <figcaption>
      Taken from DeepMind
    </figcaption>
  </figure>
</center>

The model plays against itself which is depicted above. Given a certain game state the algorithm performs a MTCS to generate the tree search probabilities $\pi$. An action is then sampled from $\pi$ which brings us in the next state. Again, perform a MTCS and so on. This goes on until the game is finished. The tree search reward $z$ is collected.


### Training

<center>
  <figure>
    <img src="images/training.png" width=800>
    <figcaption>
      Taken from DeepMind
    </figcaption>
  </figure>
</center>

$$L_{value} = (z - V)^2.$$ 

This is the squared loss induced by wrong prediction of the value functions at the experienced states.

The loss corresponding to the actions is 

$$L_{action} = - \pi^\top \log p.$$

The loss is large if the tree search probabilities $\pi$ are very different from the network probabilities $p$ and is minimized if $\pi = p$. The function $\pi$ is generated by the MCTS where the initial update probabilities $p$ are modified to account for experienced Monte-Carlo $Q$ functions and stimulate exploration by using upper confidence bounds (recall our use of these in Seminar 1 when discussing $k$-bandit problems).

The regularization loss is defined as 

$$L_{reg} = c \|\theta\|^2$$

for some constant $c$ and the current network parameters $\theta$.

The overall loss is defined as

$$L = L_{value} + L_{action} + L_{reg}.$$

## Gomoku

The implementation we use is applied to the game of [Gomoku](https://en.wikipedia.org/wiki/Gomoku).

<center>
  <figure>
    <img src="images/gomoku.jpg" width=300>
    <figcaption>
      Typical game state in Gomoku
    </figcaption>
  </figure>
</center>

Gomoku is a 2-player game where players alternatingly place stones on a square board. The aim of the game is to position $k$ consecutive stones horizontally, vertically or diagonally.

Gomoku is a much easier game than Go and board sizes up to 10 x 10 are suitable for a standard CPU to develop a good model in a few days.

Let us begin by playing a round on an 8x8 board where the target is to get 5 consecutive stones. The MCTS Model used was trained for a few days on a standard CPU.

In [1]:
from play import run

pygame 2.1.0 (SDL 2.0.16, Python 3.8.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
%%capture
run(width=8, height=8, num_consecutive=5)

The current AI good but still far from optimal: 
    
<center>
  <figure>
    <img src="images/win_against_8_8_5.png" width=300>
    <figcaption>
      Implemented Graphical User Interface: Winning against the current 8x8 and 5 consecutive stones.
    </figcaption>
  </figure>
</center>

### The network architecture 

#### Input

```python
self.board_width = board_width
self.board_height = board_height

# Define the tensorflow neural network
# 1. Input:
self.input_states = tf.placeholder(
        tf.float32, shape=[None, 4, board_width, board_height])
self.input_state = tf.transpose(self.input_states, [0, 2, 3, 1])
```

#### Network Layers

```python
# 2. Common Networks Layers
self.conv1 = tf.layers.conv2d(inputs=self.input_state,
                              filters=32, kernel_size=[3, 3],
                              padding="same", data_format="channels_last",
                              activation=tf.nn.relu)
self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=64,
                              kernel_size=[3, 3], padding="same",
                              data_format="channels_last",
                              activation=tf.nn.relu)
self.conv3 = tf.layers.conv2d(inputs=self.conv2, filters=128,
                              kernel_size=[3, 3], padding="same",
                              data_format="channels_last",
                              activation=tf.nn.relu)
```

#### Action Networks

```python
# 3-1 Action Networks
self.action_conv = tf.layers.conv2d(inputs=self.conv3, filters=4,
                                    kernel_size=[1, 1], padding="same",
                                    data_format="channels_last",
                                    activation=tf.nn.relu)
# Flatten the tensor
self.action_conv_flat = tf.reshape(
        self.action_conv, [-1, 4 * board_height * board_width])
# 3-2 Full connected layer, the output is the log probability of moves
# on each slot on the board
self.action_fc = tf.layers.dense(inputs=self.action_conv_flat,
                                 units=board_height * board_width,
                                 activation=tf.nn.log_softmax)
```

#### Evaluation Networks
 
 ```python
self.evaluation_conv = tf.layers.conv2d(inputs=self.conv3, filters=2,
                                        kernel_size=[1, 1],
                                        padding="same",
                                        data_format="channels_last",
                                        activation=tf.nn.relu)
self.evaluation_conv_flat = tf.reshape(
        self.evaluation_conv, [-1, 2 * board_height * board_width])
self.evaluation_fc1 = tf.layers.dense(inputs=self.evaluation_conv_flat,
                                      units=64, activation=tf.nn.relu)
# output the score of evaluation on current state
self.evaluation_fc2 = tf.layers.dense(inputs=self.evaluation_fc1,
                                      units=1, activation=tf.nn.tanh)
```

#### The loss function

```python
# 1. Label: the array containing if the game wins or not for each state
self.labels = tf.placeholder(tf.float32, shape=[None, 1])
# 2. Predictions: the array containing the evaluation score of each state
# which is self.evaluation_fc2
# 3-1. Value Loss function
self.value_loss = tf.losses.mean_squared_error(self.labels,
                                               self.evaluation_fc2)
# 3-2. Policy Loss function
self.mcts_probs = tf.placeholder(
        tf.float32, shape=[None, board_height * board_width])
self.policy_loss = tf.negative(tf.reduce_mean(
        tf.reduce_sum(tf.multiply(self.mcts_probs, self.action_fc), 1)))
# 3-3. L2 penalty (regularization)
l2_penalty_beta = 1e-4
vars = tf.trainable_variables()
l2_penalty = l2_penalty_beta * tf.add_n(
    [tf.nn.l2_loss(v) for v in vars if 'bias' not in v.name.lower()])
# 3-4 Add up to be the Loss function
self.loss = self.value_loss + self.policy_loss + l2_penalty

```

#### The optimizer

```python
# Define the optimizer we use for training
self.learning_rate = tf.placeholder(tf.float32)
self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.loss)

```

### Training a model

We can also train a new model based on new board dimensions. The best model encountered during training will be stored and can be used to play against.

In [4]:
import warnings

warnings.filterwarnings("ignore")

from train import TrainPipeline

training_pipeline = TrainPipeline(width=12, height=12, n_in_row=2)
training_pipeline.run()

batch i:1, episode_len:4
batch i:2, episode_len:3
batch i:3, episode_len:5
batch i:4, episode_len:3
batch i:5, episode_len:3
batch i:6, episode_len:4
batch i:7, episode_len:3
batch i:8, episode_len:3
batch i:9, episode_len:3
batch i:10, episode_len:3
current self-play batch: 10
num_playouts:1000, win: 5, lose: 5, tie:0
Found new best!
batch i:11, episode_len:3
batch i:12, episode_len:3
batch i:13, episode_len:3
batch i:14, episode_len:3
batch i:15, episode_len:3
batch i:16, episode_len:4
batch i:17, episode_len:3
batch i:18, episode_len:4
batch i:19, episode_len:3
batch i:20, episode_len:5
kl:0.13415,lr_multiplier:0.667,loss:5.691923141479492,entropy:4.891033172607422,explained_var_old:0.007,explained_var_new:-0.217
current self-play batch: 20
num_playouts:1000, win: 5, lose: 5, tie:0
batch i:21, episode_len:4
kl:0.04680,lr_multiplier:0.444,loss:5.688155174255371,entropy:4.833124160766602,explained_var_old:-0.185,explained_var_new:0.176
batch i:22, episode_len:3
kl:0.02089,lr_multiplie

## On a few technical details in the Gomoku Implementations

### Data augmentation via rotation/flipping

The game state is invariant under rotating the board or flipping the game state at one of the axes of symmetry. Hence we can simulate data based on these transformations.

```python
def get_equi_data(self, play_data):
    """augment the data set by rotation and flipping
    play_data: [(state, mcts_prob, winner_z), ..., ...]
    """
    extend_data = []
    for state, mcts_porb, winner in play_data:
        for i in [1, 2, 3, 4]:
            # rotate counterclockwise
            equi_state = np.array([np.rot90(s, i) for s in state])
            equi_mcts_prob = np.rot90(np.flipud(
                mcts_prob.reshape(self.board_height, self.board_width)), i)
            extend_data.append((equi_state,
                                np.flipud(equi_mcts_prob).flatten(),
                                winner))
            # flip horizontally
            equi_state = np.array([np.fliplr(s) for s in equi_state])
            equi_mcts_prob = np.fliplr(equi_mcts_prob)
            extend_data.append((equi_state,
                                np.flipud(equi_mcts_prob).flatten(),
                                winner))
    return extend_data
```